In [2]:
pip install imgaug

     ------------------------------------- 948.0/948.0 kB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 12.5 MB/s eta 0:00:00
     --------------------------------------- 38.6/38.6 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.0
    Uninstalling scipy-1.10.0:
      Successfully uninstalled scipy-1.10.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.


In [44]:
import imgaug.augmenters as iaa
import numpy as np
import cv2

# Load image
img = cv2.imread(r"C:\Users\annem\OneDrive\Desktop\photos\George Hanson\Hanson George.jpg")

# Define augmentation sequence
seq = iaa.Sequential([
    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16 pixels
    iaa.Fliplr(0.5),
    iaa.Fliplr(1.0),
   # iaa.Flipud(1.0),
    # flip horizontally with probability 0.5
    #iaa.GaussianBlur(sigma=(0, 3.0)), # apply gaussian blur with sigma between 0 and 3.0
    #iaa.Dropout(p=(0, 0.2)), # randomly remove up to 20% of the pixels
    #iaa.AdditiveGaussianNoise(scale=(0, 0.1*255)), # add gaussian noise with mean 0 and standard deviation between 0 and 0.1*255
    iaa.LinearContrast((0.75, 1.5)), # adjust contrast by scaling each pixel value between 0.75 and 1.5
], random_order=True) # apply augmentations in random order

# Generate augmented images
f=r"C:\Users\annem\OneDrive\Desktop\photos\George Hanson"
images_aug = [seq.augment_image(img) for _ in range(10)]

# Save augmented images
for i, image_aug in enumerate(images_aug):
    cv2.imwrite(f"{f}/image_aug__{i}.jpg", image_aug)


In [15]:
import os
desired_size = (256, 256) # for example

for filename in os.listdir(r'C:\Users\annem\OneDrive\Desktop\photos\t\Edith Hanson Storms'):
    img = cv2.imread(os.path.join(r'C:\Users\annem\OneDrive\Desktop\photos\t\Edith Hanson Storms', filename))
    if img is not None:
        old_size = img.shape[:2]
        ratio = float(desired_size[0])/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])
        # resize the image
        img = cv2.resize(img, (new_size[1], new_size[0]))
        # create a new image with the desired size and fill it with white pixels
        new_img = np.ones((desired_size[0], desired_size[1], 3), dtype=np.uint8) * 255
        # calculate the center offset of the image
        center_offset = ((desired_size[0]-new_size[0])//2, (desired_size[1]-new_size[1])//2)
        # copy the resized image into the center of the new image
        new_img[center_offset[0]:center_offset[0]+new_size[0], center_offset[1]:center_offset[1]+new_size[1]] = img
        # save the new image with the same filename in the folder
        cv2.imwrite(os.path.join('C:/Users/annem/OneDrive/Desktop/photos/t/Edith Hanson Storms/', filename), new_img)


In [27]:
f=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Edith Hanson Storms\1915~Edith WWSII Till WWSI.jpg_face2.jpg")
cv2.imshow('im',f)

# CNN Main Code

In [1]:
# Import necessary libraries
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.models import Sequential
from tensorflow.keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
import pickle

# Define function to load dataset
def load_dataset(path):
    X, y = [], []
    for foldername in os.listdir(path):
        folderpath = os.path.join(path, foldername)
        for filename in os.listdir(folderpath):
            imgpath = os.path.join(folderpath, filename)
            img = cv2.imread(imgpath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
            img = cv2.resize(img, (256, 256))
           # print(img.shape)# Resize image
            X.append(img)
            y.append(foldername)
           #     # Convert data to numpy arrays
    X = np.array(X)
    y = np.array(y)
    # Encode labels
    encoder = LabelEncoder()
    y = encoder.fit_transform(y)
    y = np_utils.to_categorical(y)
    # Save the encoder to a file
  #  encoder.save('encoder.h5')
    with open('encoder.pkl', 'wb') as f:
        pickle.dump(encoder, f)
    # Return data and labels
    
    return X, y

# # Load dataset"C:\Users\annem\OneDrive\Desktop\photos\train"
X_train, y_train = load_dataset(r'C:\Users\annem\OneDrive\Desktop\cnn\train')
X_test, y_test = load_dataset(r'C:\Users\annem\OneDrive\Desktop\cnn\test')

# Define CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Save the model to a file
# with open('model.pkl', 'wb') as f:
#     pickle.dump(model, f)
#model.save('model.h5')

C:\Users\annem\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
9/9 [==============================] - 6s 606ms/step - loss: 382.4240 - accuracy: 0.0687 - val_loss: 5.9296 - val_accuracy: 0.1159
Epoch 2/50
9/9 [==============================] - 5s 568ms/step - loss: 3.9288 - accuracy: 0.2176 - val_loss: 1.6734 - val_accuracy: 0.7246
Epoch 3/50
9/9 [==============================] - 5s 586ms/step - loss: 1.9772 - accuracy: 0.4695 - val_loss: 0.9819 - val_accuracy: 0.8696
Epoch 4/50
9/9 [==============================] - 4s 459ms/step - loss: 0.7975 - accuracy: 0.8130 - val_loss: 0.6378 - val_accuracy: 0.8406
Epoch 5/50
9/9 [==============================] - 4s 479ms/step - loss: 0.7964 - accuracy: 0.8435 - val_loss: 0.4188 - val_accuracy: 0.8986
Epoch 6/50
9/9 [==============================] - 5s 524ms/step - loss: 0.7662 - accuracy: 0.8397 - val_loss: 0.4495 - val_accuracy: 0.8841
Epoch 7/50
9/9 [==============================] - 4s 464ms/step - loss: 1.4924 - accuracy: 0.7672 - val_loss: 0.4790 - val_accuracy: 0.8986
Epoch 8/50
9/9 [==

In [10]:
layer_outputs=[layer.output for layer in model.layers[1:]]
print(layer_outputs)

[<KerasTensor: shape=(None, 127, 127, 32) dtype=float32 (created by layer 'max_pooling2d')>, <KerasTensor: shape=(None, 125, 125, 64) dtype=float32 (created by layer 'conv2d_1')>, <KerasTensor: shape=(None, 62, 62, 64) dtype=float32 (created by layer 'max_pooling2d_1')>, <KerasTensor: shape=(None, 60, 60, 128) dtype=float32 (created by layer 'conv2d_2')>, <KerasTensor: shape=(None, 30, 30, 128) dtype=float32 (created by layer 'max_pooling2d_2')>, <KerasTensor: shape=(None, 115200) dtype=float32 (created by layer 'flatten')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'dense')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'dropout')>, <KerasTensor: shape=(None, 17) dtype=float32 (created by layer 'dense_1')>]


In [3]:
pip install opencv-python

  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Load the trained model and label encoder
#model = pickle.load(open('model.pkl', 'rb'))
from tensorflow.keras.models import load_model
import pickle
import cv2
import numpy as np
#model=load_model('model.h5')
le = pickle.load(open('encoder.pkl', 'rb'))
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\from_SD\1915~ Ede WWSII Paul Mathilda in Washington Park.jpg")
# Predict the name of the person in a test image
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\cnn\train\Winslow Page Storms\Winslow Page Storms  from Storms_album.jpg")
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\from_SD\Schwartz Anna and Chris her son Storms_album.jpg")
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\from_SD\photo1 (10).jpg")
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\from_SD\1921 11 WWS p1 Death Claims Pythian.jpg")
#img = cv2.imread(r'C:\Users\annem\OneDrive\Desktop\Face Recognition\sd\#44_WashingtonPark_Fall_1913.jpg')
img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\sd\#55 Edith WWII Til Uncle Paul (2).jpg")
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\from_SD\#63 1911 WWStorms Election Ad.jpg")
##img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\photos\train\William Wallace storms\wwstorms.jpg")
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\from_SD\1939~Elinor Jean Storms possibly a college portrait.jpg")
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\from_SD\#63 1911 WWStorms Election Ad.jpg")
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\from_SD\1892~ W W Storms portrat by Ridgeway - Madison.jpg")
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\photos\train\Ana Marie Thompson Hanson\image_aug_3.jpg")
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\from_SD\Winslow Page Storms and Melissa Meacham Storms Storms_album.jpg")
#img=cv2.imread(r"C:\Users\annem\OneDrive\Desktop\Face Recognition\from_SD\1920~ Paul Matson.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#img_copy=img.copy()
#img_copy = cv2.resize(img_copy, (256, 256))
#img = cv2.resize(img, (500, 500))
#img = np.expand_dims(img, axis=0)
#pred = model.predict(img)
##pred_label = np.argmax(pred)
#name=le.inverse_transform([pred_label])[0]
# Draw bounding box around detected face
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
names=[]
#faces=cv2.resize(faces, (256,256))
#print(len(faces))
for (x,y,w,h) in faces:
    # Extract the face ROI and preprocess it for the model
    roi = gray[y:y+h, x:x+w]
    roi = cv2.resize(roi, (256, 256))
   
    roi = cv2.cvtColor(roi, cv2.COLOR_GRAY2RGB)
    #roi = np.expand_dims(roi, axis=0)
    #roi = np.expand_dims(roi, axis=3)
    roi = roi / 255.0
    #print(roi.shape)
    # Perform prediction for the face ROI
    pred = model.predict(np.expand_dims(roi, axis=0))
    pred_label = np.argmax(pred)
    name = le.inverse_transform([pred_label])[0]
    names.append(name)
    # Draw bounding box and label for the face
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.putText(img, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

# Display the image with bounding boxes and labels for each face
#im=cv2.resize(img, (250,250))
cv2.imshow('Image', img)
print(names)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 [==============================] - 0s 35ms/step
['Ed Beck', 'Ed Beck', 'Ed Beck']


In [2]:
from tensorflow.keras.models import load_model
# Load the trained model and label encoder
#model = pickle.load(open('model.pkl', 'rb'))
model=load_model('model.h5')
le = pickle.load(open('encoder.pkl', 'rb'))
# Predict the name of the person in a test image
img = cv2.imread(r'C:\Users\annem\OneDrive\Desktop\Face Recognition\sd\#44_WashingtonPark_Fall_1913.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_copy=img.copy()
#img = cv2.resize(img, (256, 256))
img = np.expand_dims(img, axis=0)
pred = model.predict(img)
pred_label = np.argmax(pred)
name=le.inverse_transform([pred_label])[0]
# Draw bounding box around detected face
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
gray = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
faces = cv2.resize(faces, (256, 256))
for (x,y,w,h) in faces:
    cv2.rectangle(img_copy,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.putText(img_copy, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

# Display image with name and bounding box
cv2.imshow('Image', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()


NameError: name 'pickle' is not defined

In [17]:
# Load the trained model and label encoder
#model = pickle.load(open('model.pkl', 'rb'))
model=load_model('model.h5')
le = pickle.load(open('encoder.pkl', 'rb'))

# Load the test image
#img = cv2.imread(r"C:\Users\annem\OneDrive\Desktop\photos\train\Anna Caspersen Schwartz Thompson\Anna Caspersen Schwartz Thompson.jpg")
img = cv2.imread(r'C:\Users\annem\OneDrive\Desktop\Face Recognition\sd\#44_WashingtonPark_Fall_1913.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_copy = img.copy()

# Detect faces in the image
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
gray = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

# Iterate through the detected faces and perform prediction for each face
for (x,y,w,h) in faces:
    # Extract the face ROI and preprocess it for the model
    roi = gray[y:y+h, x:x+w]
    roi = cv2.resize(roi, (64, 64))
    roi = cv2.cvtColor(roi, cv2.COLOR_GRAY2RGB)
    roi = np.expand_dims(roi, axis=0)
    roi = np.expand_dims(roi, axis=3)
    roi = roi / 255.0
    
    # Perform prediction for the face ROI
    pred = model.predict(np.expand_dims(roi, axis=0))
    pred_label = np.argmax(pred)
    name = le.inverse_transform([pred_label])[0]
    
    # Draw bounding box and label for the face
    cv2.rectangle(img_copy,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.putText(img_copy, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

# Display the image with bounding boxes and labels for each face
cv2.imshow('Image', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()


ValueError: in user code:

    File "C:\Users\annem\anaconda3\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\annem\anaconda3\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\annem\anaconda3\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\annem\anaconda3\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\Users\annem\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\annem\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Inp

In [22]:
img1 = cv2.imread(r"C:\Users\annem\OneDrive\Desktop\photos\train\Anna Caspersen Schwartz Thompson\Anna Caspersen Schwartz Thompson.jpg")
img2 = cv2.imread(r'C:\Users\annem\OneDrive\Desktop\Face Recognition\sd\#44_WashingtonPark_Fall_1913.jpg')
img1.shape

(612, 493, 3)

In [ ]:
(1168, 1517, 3)

In [10]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.models import Sequential
from tensorflow.keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
import pickle
# Load the trained model and label encoder
model = load_model('model.h5')
le = pickle.load(open('encoder.pkl', 'rb'))

# Load the image
img = cv2.imread(r'C:\Users\annem\OneDrive\Desktop\Face Recognition\sd\#44_WashingtonPark_Fall_1913.jpg')

# Convert image to grayscale and detect faces
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# #img_copy=img.copy()
# img = cv2.resize(img, (64, 64))
# img = np.expand_dims(img, axis=0)
faces = face_cascade.detectMultiScale(img, 1.3, 5)

# Loop over detected faces and classify each one
for (x, y, w, h) in faces:
    # Extract the ROI where the face is located
    roi = gray[y:y+h, x:x+w]
    # Resize the ROI to match the input size of the model
    roi = cv2.resize(roi, (64, 64))
    roi = cv2.cvtColor(roi, cv2.COLOR_GRAY2RGB)
    # Apply histogram equalization to improve contrast
    roi = cv2.equalizeHist(roi)
    # Reshape the ROI to have a single channel (grayscale)
    roi = np.expand_dims(roi, axis=-1)
    # Reshape the ROI to have a batch dimension (needed for model.predict)
    roi = np.expand_dims(roi, axis=0)
    # Normalize the pixel values to be between 0 and 1
    roi = roi / 255.0
    # Apply the trained model to the ROI to classify the person
    pred = model.predict(roi)
    pred_label = np.argmax(pred)
    name = le.inverse_transform([pred_label])[0]
    # Draw a bounding box around the face and display the name
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(img, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# Display the image with bounding boxes and names
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()



error: OpenCV(4.7.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<1,-1,-1>,struct cv::impl::A0x981fb336::Set<3,4,-1>,struct cv::impl::A0x981fb336::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 3


In [45]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the directories for the training data
train_dir = r'C:\Users\annem\OneDrive\Desktop\photos'
#validation_dir = 'path/to/validation/directory'
test_dir = r'C:\Users\annem\OneDrive\Desktop\photos'

# Set up the data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Set up the image generators
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    classes=['Ana Marie Thomspson Hanson', 'Ana Caspersen Schwartz Thompson', 'Aunt Kate', 'Chris Schwarz', 'Ed Beck', 'Edith Hanson Storms', 'Eleanor Nora Storms', 'Elinor Jean Storms Danford', 'Florence Penn', 'George Hanson', 'Hans Isaac Hanson', 'Paul Matson', 'Persis Melissa Meacham Storms', 'william wallace', 'William Wallace storms', 'William Wallace Storms II', 'Winslow Page Storms']
)



test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    classes=['Ana Marie Thomspson Hanson', 'Ana Caspersen Schwartz Thompson', 'Aunt Kate', 'Chris Schwarz', 'Ed Beck', 'Edith Hanson Storms', 'Eleanor Nora Storms', 'Elinor Jean Storms Danford', 'Florence Penn', 'George Hanson', 'Hans Isaac Hanson', 'Paul Matson', 'Persis Melissa Meacham Storms', 'william wallace', 'William Wallace storms', 'William Wallace Storms II', 'Winslow Page Storms']
)


Found 202 images belonging to 17 classes.
Found 202 images belonging to 17 classes.


In [47]:
from tensorflow.keras import models
from tensorflow.keras import layers
import numpy as np
import os
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])




NameError: name 'img_width' is not defined